In [1]:
import glob,os,sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
import re
import scipy.stats as st
from os.path import expanduser
import cmath
import scipy.optimize as op
import time
import gzip
import pandas as pd
from sympy import Symbol
from sympy.solvers import solve
from matplotlib.colors import LogNorm

%matplotlib inline

In [3]:
home = os.path.expanduser("~")

direc = os.listdir(".")

In [4]:
name = 'alllc_full_info2.CSV'  
#Library/Mobile Documents/com~apple~CloudDocs/Microlensing/OSU trip/Matt/result_file/
tempdata = home+'/Desktop/trial_runs/'+str(name)
#&((df['f_s_true']*( (2 + df['u0_true']**2) / (df['u0_true']*np.sqrt(4 + df['u0_true']**2)) ) + (1-df['f_s_true']) )>1.5) & (df['u0_true']>0.1)
df = pd.read_csv(tempdata)

df['u0_true'] = np.abs(df['u0_true'])
df['u0_fitted'] = np.abs(df['u0_fitted'])

In [5]:
print len(df)
#df = df[ (df['chi_2_2']>-25000) & (df['s_fitted']<5) & ((df['f_s_true']*( (2 + df['u0_true']**2) / (df['u0_true']*np.sqrt(4 + df['u0_true']**2)) ) + (1-df['f_s_true']) )>1.1)]
#df = df[(np.abs(df['t0_fitted']-df['tp_fitted'])>1) & ((-2*df['chi_2_2']/41039)>1.003) ]
df_org = df
df['q_fitted_log'][df['q_fitted_log']>1] = np.log10( 1/df['q_fitted'][df['q_fitted_log']>0] )
df['q_fitted'][df['q_fitted_log']>1] = ( 1/df['q_fitted'][df['q_fitted_log']>0] )

13029


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [21]:
G = 6.67e-11
m_sun = 1.989e30
kpc = 3.086e19
c = 2.99e8
const = 4*(G*m_sun)/(kpc*(c**2))
d_rel = (df['D_l']*df['D_s'])/(df['D_s']-df['D_l'])
df['theta_E'] = np.sqrt(df['m_l']*const / d_rel)

In [23]:
df['mu_l'] = np.sqrt((df['mu_l_b']**2)+(df['mu_l_l']**2))
df['mu_s'] = np.sqrt((df['mu_s_b']**2)+(df['mu_s_l']**2))

In [24]:
df['mu_rel'] =  df['mu_s']- df['mu_l']

In [25]:
df['weight'] = df['mu_rel']*df['theta_E']

In [39]:
(len(df[df['u0_fitted']>0.045])/float(len(df)))*100

76.08412003991096

In [36]:
np.sum(df['weight'][df['u0_fitted']<0.045])

0.0002350604436142973

In [38]:
(np.sum(df['weight'][df['u0_fitted']>0.045])/np.sum(df['weight']))*100

76.1795135423112

In [40]:
red_chi_2_1 = np.abs(2*df['chi_2_1'])
red_chi_2_2 = np.abs(2*df['chi_2_2'])
red_chi_subt = red_chi_2_1 - red_chi_2_2

In [42]:
(len(df[-2*df['chi_2_2']>46024])/float(len(df)))*100

7.590759075907591

In [43]:
(np.sum(df['weight'][-2*df['chi_2_2']>46024])/np.sum(df['weight']))*100

7.54489495980817

In [44]:
(len(df[red_chi_subt<70])/float(len(df)))*100

40.29472714713332

In [45]:
(np.sum(df['weight'][red_chi_subt<70])/np.sum(df['weight']))*100

40.4494583938784

In [50]:
def weighted_med (true,fitted,weight):
    temp = (weight/np.sum(weight))*(fitted - true)
    return (np.median(np.abs(temp-np.median(temp))))

def med_med (true,fitted):
    temp = fitted - true
    return (np.median(np.abs(temp-np.median(temp))))


In [51]:
med_med(df['s_true'],df['s_fitted'])

0.2681770546900001

In [52]:
weighted_med(df['s_true'],df['s_fitted'],df['weight'])

1.9324444410393143e-05